In [ ]:
# !pip install transformers
# !wget https://github.com/t-davidson/hate-speech-and-offensive-language/raw/master/data/labeled_data.csv
# Put me under data/
!pip install ekphrasis
!pip3 install nltk emoji

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
# from tqdm import tqdm
from tqdm.notebook import tqdm
import pandas as pd
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.utils import shuffle
import sklearn
import random
from ekphrasis.classes.spellcorrect import SpellCorrector
import re
import emoji as emoji
import string
from sklearn.metrics import classification_report, accuracy_score
seed = 888
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# from https://github.com/ZeroxTM/BERT-CNN-Fine-Tuning-For-Hate-Speech-Detection-in-Online-Social-Media
def pre_process_dataset(values):
    new_values = list()

    emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '</3', ':\*',
                 ';-)',
                 ';)', ';-D', ';D', '(;', '(-;', ':-(', ':(', '(:', '(-:', ':,(', ':\'(', ':"(', ':((', ':D', '=D',
                 '=)',
                 '(=', '=(', ')=', '=-O', 'O-=', ':o', 'o:', 'O:', 'O:', ':-o', 'o-:', ':P', ':p', ':S', ':s', ':@',
                 ':>',
                 ':<', '^_^', '^.^', '>.>', 'T_T', 'T-T', '-.-', '*.*', '~.~', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp',
                 ':-|',
                 ':->', ':-<', '$_$', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X']

    for value in values:
        text = value.replace(".", " ").lower()
        text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
        users = re.findall("[@]\w+", text)
        for user in users:
            text = text.replace(user, "<user>")
        urls = re.findall(r'(https?://[^\s]+)', text)
        if len(urls) != 0:
            for url in urls:
                text = text.replace(url, "<url >")
        for emo in text:
            if emo in emoji.UNICODE_EMOJI:
                text = text.replace(emo, "<emoticon >")
        for emo in emoticons:
            text = text.replace(emo, "<emoticon >")
        numbers = re.findall('[0-9]+', text)
        for number in numbers:
            text = text.replace(number, "<number >")
        text = text.replace('#', "<hashtag >")
        text = re.sub(r"([?.!,¿])", r" ", text)
        text = "".join(l for l in text if l not in string.punctuation)
        text = re.sub(r'[" "]+', " ", text)
        new_values.append(text)
    return new_values

In [ ]:
sp = SpellCorrector(corpus="english") 
print(sp.correct("korrect"))

In [ ]:

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
   
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    spell_correction=True,
    remove_tags=True,
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
    
)

sentences = [
    "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
    "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
    "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!   YAAAAAAY !!! :-D http://sentimentsymposium.com/."
]

for s in sentences:
    print(" ".join(text_processor.pre_process_doc(s)))

In [ ]:
# path of data and the name of pretrained weights
path1 = '../input/nlpproj/labeled_data.csv'
path = '../input/nlpproj/labelled_data_spell.csv'
model_name = "vinai/bertweet-base"

In [ ]:
# df = pd.read_csv(path, index_col = 0)
df1 = pd.read_csv(path)
df = df1.dropna().reset_index()

df2 = pd.read_csv(path1)
df = df2[df1['tweet'].isna()==False].dropna().reset_index()

df = shuffle(df)
df.describe()

In [ ]:
# 364 & 11176, Just to make sure the experiment is reproducible
df.iloc[0:2] 

In [ ]:
df['class'].value_counts()

In [ ]:
train_data = df.sample(frac = 0.8)
test_data = df.drop(train_data.index)
valid_data = test_data.sample(frac = 0.5)
test_data = test_data.drop(valid_data.index)

display(train_data.head())
print("===================================")
display(valid_data.head())
print("===================================")
display(test_data.head())

print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

# Build Dataset

The data are stored as a CSV and as a pickled pandas dataframe (Python 2.7). Each data df contains 5 columns:

count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

hate_speech = number of CF users who judged the tweet to be hate speech.

offensive_language = number of CF users who judged the tweet to be offensive.

neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, model_name,  train = True, device='cuda'):
        super(Dataset, self).__init__()
        self.df = df
        self.device = device
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        device = self.device
        count = torch.LongTensor([self.df.iloc[idx]['count']])
        hate_speech = torch.LongTensor([self.df.iloc[idx]['hate_speech']])
        offensive_language = torch.LongTensor([self.df.iloc[idx]['offensive_language']])
        neither = torch.LongTensor([self.df.iloc[idx]['neither']])
        target = torch.LongTensor([self.df.iloc[idx]['class']])
        tweet = self.df.iloc[idx]['tweet']
        return (count.to(device), hate_speech.to(device), offensive_language.to(device)
                , neither.to(device), target.to(device), tweet)
        
        

In [ ]:
class DatasetSample(torch.utils.data.Dataset):
    def __init__(self, df, model_name,  train = True, device='cuda'):
        super(DatasetSample, self).__init__()
        self.df = df
        self.device = device
        self.cls0 = self.df.loc[df['class'] ==0]
        self.cls1 = self.df.loc[df['class'] ==1]
        self.cls2 = self.df.loc[df['class'] ==2]
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        zero = random.randint(0, len(self.df.loc[df['class'] ==0])-1)
        one = random.randint(0, len(self.df.loc[df['class'] ==1])-1)
        two = random.randint(0, len(self.df.loc[df['class'] ==2])-1)
        tweetHate = self.cls0.iloc[zero]['tweet']
        tweetOffen = self.cls1.iloc[one]['tweet']
        tweetNeither = self.cls2.iloc[two]['tweet']
        return (tweetHate, tweetOffen, tweetNeither)
        
        

In [ ]:
tweetHate =  len(df.loc[df['class'] ==0])
tweetOffen = len(df.loc[df['class'] ==1])
tweetNeither = len(df.loc[df['class'] ==2])
print(tweetHate)
print(tweetOffen)
print(tweetNeither)

In [ ]:
dataset = Dataset(df, model_name, device=device)
dataloader = torch.utils.data.DataLoader(dataset, batch_size= 6, shuffle=True)
datasetSample = DatasetSample(df, model_name, device=device)
dataloadersample = torch.utils.data.DataLoader(datasetSample, batch_size= 6, shuffle=True)
output = next(iter(dataloader))
print(list(output[-1]))
sentences = list(output[-1])
print("==================================")
slist = []
for s in sentences:
    slist.append(" ".join(text_processor.pre_process_doc(s)))
print(slist)

In [ ]:
output = next(iter(dataloadersample))
print(output)

# Utils

In [ ]:
def number_params(model, exclude_freeze=False):
    """calculate the number of parameters in a model

    Args:
        model (nn.Module): PyTorch model
        exclude_freeze (bool, optional): Whether to count the frozen layer. Defaults to False.
    """
    pp = 0
    for p in list(model.parameters()):
        if exclude_freeze and p.requires_grad is False:
            continue
        nn = 1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

# Model

In [ ]:
class LastAttnModelForOrdinalReg(nn.Module):
    """
    Use the [CLS] as query and all other output as key and values.
    Pass it to a Multi-Head Attention, then a Linear classifier
    
    Args:
      auxiliary_head(list(int)): Only used when training
        - list of idx of hidden_layers that will be used as auxiliary_head. Here `idx` start from 1
        - See BertConfig['num_hidden_layers'] for total number of layers
        - EG: `auxiliary_head=[10,11,12]`.
      last_hidden_layer(int): Treat the output of this layer as last_hidden_layer

    Examples:
      tweet = iter(dataloader_train).next()[-1]
      modelA = LastAttnModel(pretrain_model, tokenizer).to(device)
      out, attn_weight = modelA(list(tweet))
      print(out.shape, attn_weight.shape) #torch.Size([32, 3]) torch.Size([32, 1, 49])
    """
    
    def __init__(self, pretrain_model, tokenizer, 
                 last_attn_num_head = 8,
                 classifier_hidden_dim = 512, 
                 classifier_dropout = 0,
                 num_labels = 3, 
                 freeze_pretrained=False,
                 auxiliary_head=None,
                 last_hidden_layer=-1):
        super(LastAttnModelForOrdinalReg, self).__init__()
        self.pretrain_model = pretrain_model
        self.tokenizer = tokenizer
        self.auxiliary_head = auxiliary_head
        self.num_layers = len(pretrain_model.encoder.layer)
        self.last_hidden_layer = last_hidden_layer
        
        if freeze_pretrained:
            if self.auxiliary_head is not None:
              warnings.warn("freeze_pretrained and auxiliary_head set to True together is useless for training. Consider use `finetune()`")
            print("You are freezing the BERT pertrain")
            for name, p in self.pretrain_model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
        
        embed_size = pretrain_model.embeddings.word_embeddings.embedding_dim
        self.last_attn = nn.MultiheadAttention(embed_size, last_attn_num_head, batch_first=True)
        self.final_classifier = nn.Sequential(
            nn.Linear(embed_size, classifier_hidden_dim),
            nn.ReLU(),
            nn.Dropout(classifier_dropout),
            nn.Linear(classifier_hidden_dim, 1)
        )
        self.linear_1_bias = nn.Parameter(torch.zeros(2).float())
        # deal with aux 
        if self.auxiliary_head is not None:
          self.aux_classifiers = nn.ModuleList()
          for i in self.auxiliary_head:
            self.aux_classifiers.append(nn.Sequential(
              nn.Linear(embed_size, classifier_hidden_dim),
              nn.ReLU(),
              nn.Dropout(classifier_dropout),
              nn.Linear(classifier_hidden_dim, classifier_hidden_dim)
            ))


        print(f"Total number of params: {number_params(self)}")
        print(f"Total number of trainable params: {number_params(self, exclude_freeze=True)}")
    def forward(self, src, has_mask=False):
        # print(src)
        out = []
        tokens = self.tokenizer(src, padding=True, truncation=True, max_length=50)
        inputs = torch.LongTensor(tokens['input_ids']).to(device)
        if has_mask == True:
            # attention_mask=(inputs != 0).float() # here `0` is the <pad> token, i guess
            attention_mask = torch.LongTensor(tokens['attention_mask']).to(device)
            pre_train_output = self.pretrain_model(inputs, attention_mask=attention_mask)
        else:
            pre_train_output = self.pretrain_model(inputs)
        # last_hidden = pre_train_output["last_hidden_state"]      # (N,T,E)
        last_hidden = pre_train_output["hidden_states"][self.last_hidden_layer]
        last_hidden_state_cls = last_hidden[:, 0, :].unsqueeze(1)   # (N,1,E)
        last_hidden_state_rest = last_hidden[:, 1:, :] # (N,T-1,E)
        atten_mask_pad = (inputs == 0)[:,1:] #(N,T-1)
        last_attn_out, last_attn_w = self.last_attn(last_hidden_state_cls, last_hidden_state_rest, last_hidden_state_rest,
                                                    key_padding_mask=atten_mask_pad) #(N,1,E), (N,1,T-1)
        last_attn_out = last_attn_out.squeeze(1) #(N,E)
        output = self.final_classifier(last_attn_out)
        out += [output, last_attn_w]
        ## auxiliary_head
        if self.auxiliary_head is not None:
          if "hidden_states" not in pre_train_output:
            raise Exception("Put `pre_train_output=True` in AutoConfig")
          for idx in range(len(self.aux_classifiers)):
            hidden_cls = pre_train_output["hidden_states"][self.auxiliary_head[idx-1]][:, 0, :] # (N,E)
            out.append(self.aux_classifiers[idx](hidden_cls))
        out = out[0]
        out = out + self.linear_1_bias
        
        return out, torch.sigmoid(out)
    
# ## Usage
# tweet = iter(dataloader_train).next()[-1]
# modelA = LastAttnModel(pretrain_model, tokenizer).to(device)
# out, attn_weight = modelA(list(tweet))
# print(out.shape, attn_weight.shape)

In [ ]:
class LanguageModelCNNForContrastive(nn.Module):
    
    def __init__(self, model_name, num_labels = 3, freeze_pretrained=False,):
        super(LanguageModelCNNForContrastive, self).__init__()
        self.model = AutoModel.from_pretrained(model_name, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, normalization=True)
        
        self.conv = nn.Conv2d(in_channels=13, out_channels=13, kernel_size=(3, 768), padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(624, 624) # before : 442 with max_length 36 # 806 with max_length 64
        self.flat = nn.Flatten()
        self.softmax = nn.LogSoftmax(dim=1)
        if freeze_pretrained:
            print("You are freezing the BERT")
            for name, p in self.model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
                if '11' in name:
                     p.requires_grad = True
                if '10' in name:
                     p.requires_grad = True
                if '9' in name:
                     p.requires_grad = True
        print(f"Total number of params: {number_params(self.model)}")
        print(f"Total number of trainable params: {number_params(self.model, exclude_freeze=True)}")

    def forward(self, src, has_mask=False):
        # print(src)
        slist = []
        for s in src:
            slist.append(" ".join(text_processor.pre_process_doc(s)))
        output = self.tokenizer(slist, padding='max_length', truncation=True, max_length=50)
        output = torch.LongTensor(output['input_ids']).to(device)
        if has_mask == True:
            attention_mask=(output != 0).float() # here `0` is the <pad> token, i guess
            output = self.model(output, attention_mask=attention_mask, output_hidden_states=True)
        else:
            output = self.model(output, output_hidden_states=True)
        
        out = output['hidden_states']
        
        x = torch.transpose(torch.cat(tuple([t.unsqueeze(0) for t in out]), 0), 0, 1)
        # print(x.size())# 32 13 50 768
        x = self.pool(self.dropout(self.relu(self.conv(self.dropout(x)))))
        x = self.dropout(self.flat(self.dropout(x)))
        #print(x.size())
        x = self.fc(x)
        
        return x

In [ ]:
class LastAttnModelForContrastive(nn.Module):
    """
    Use the [CLS] as query and all other output as key and values.
    Pass it to a Multi-Head Attention, then a Linear classifier
    
    Args:
      auxiliary_head(list(int)): Only used when training
        - list of idx of hidden_layers that will be used as auxiliary_head. Here `idx` start from 1
        - See BertConfig['num_hidden_layers'] for total number of layers
        - EG: `auxiliary_head=[10,11,12]`.
      last_hidden_layer(int): Treat the output of this layer as last_hidden_layer

    Examples:
      tweet = iter(dataloader_train).next()[-1]
      modelA = LastAttnModel(pretrain_model, tokenizer).to(device)
      out, attn_weight = modelA(list(tweet))
      print(out.shape, attn_weight.shape) #torch.Size([32, 3]) torch.Size([32, 1, 49])
    """
    
    def __init__(self, pretrain_model, tokenizer, 
                 last_attn_num_head = 8,
                 classifier_hidden_dim = 512, 
                 classifier_dropout = 0,
                 num_labels = 3, 
                 freeze_pretrained=False,
                 auxiliary_head=None,
                 last_hidden_layer=-1):
        super(LastAttnModelForContrastive, self).__init__()
        self.pretrain_model = pretrain_model
        self.tokenizer = tokenizer
        self.auxiliary_head = auxiliary_head
        self.num_layers = len(pretrain_model.encoder.layer)
        self.last_hidden_layer = last_hidden_layer
        # self.stage1 = stage1

        if freeze_pretrained:
            if self.auxiliary_head is not None:
              warnings.warn("freeze_pretrained and auxiliary_head set to True together is useless for training. Consider use `finetune()`")
            print("You are freezing the BERT pertrain")
            for name, p in self.pretrain_model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
#                 if '11' in name:
#                      p.requires_grad = True
#                 if '10' in name:
#                      p.requires_grad = True
#                 if '9' in name:
#                      p.requires_grad = True
        
        embed_size = pretrain_model.embeddings.word_embeddings.embedding_dim
        self.last_attn = nn.MultiheadAttention(embed_size, last_attn_num_head, batch_first=True)
        self.final_classifier = nn.Sequential(
            nn.Linear(embed_size, classifier_hidden_dim),
            nn.ReLU(),
            nn.Dropout(classifier_dropout),
            nn.Linear(classifier_hidden_dim, 20)
        )
        self.stageone = nn.Sequential(nn.Linear(20, 20), nn.ReLU(), nn.Linear(20, num_labels))
        # deal with aux 
        if self.auxiliary_head is not None:
          self.aux_classifiers = nn.ModuleList()
          for i in self.auxiliary_head:
            self.aux_classifiers.append(nn.Sequential(
              nn.Linear(embed_size, classifier_hidden_dim),
              nn.ReLU(),
              nn.Dropout(classifier_dropout),
              nn.Linear(classifier_hidden_dim, classifier_hidden_dim)
            ))


        print(f"Total number of params: {number_params(self)}")
        print(f"Total number of trainable params: {number_params(self, exclude_freeze=True)}")
    def forward(self, src, has_mask=False, stage1 = False, nograd = False):
        # print(src)
        if nograd == True:
            with torch.no_grad():
                out = []
                tokens = self.tokenizer(src, padding=True, truncation=True, max_length=50)
                inputs = torch.LongTensor(tokens['input_ids']).to(device)
                if has_mask == True:
                    # attention_mask=(inputs != 0).float() # here `0` is the <pad> token, i guess
                    attention_mask = torch.LongTensor(tokens['attention_mask']).to(device)
                    pre_train_output = self.pretrain_model(inputs, attention_mask=attention_mask)
                else:
                    pre_train_output = self.pretrain_model(inputs)
                # last_hidden = pre_train_output["last_hidden_state"]      # (N,T,E)
                last_hidden = pre_train_output["hidden_states"][self.last_hidden_layer]
                last_hidden_state_cls = last_hidden[:, 0, :].unsqueeze(1)   # (N,1,E)
                last_hidden_state_rest = last_hidden[:, 1:, :] # (N,T-1,E)
                atten_mask_pad = (inputs == 0)[:,1:] #(N,T-1)
                last_attn_out, last_attn_w = self.last_attn(last_hidden_state_cls, last_hidden_state_rest, last_hidden_state_rest,
                                                            key_padding_mask=atten_mask_pad) #(N,1,E), (N,1,T-1)
                last_attn_out = last_attn_out.squeeze(1) #(N,E)
                output = self.final_classifier(last_attn_out)
                out += [output, last_attn_w]
                ## auxiliary_head
                if self.auxiliary_head is not None:
                  if "hidden_states" not in pre_train_output:
                    raise Exception("Put `pre_train_output=True` in AutoConfig")
                  for idx in range(len(self.aux_classifiers)):
                    hidden_cls = pre_train_output["hidden_states"][self.auxiliary_head[idx-1]][:, 0, :] # (N,E)
                    out.append(self.aux_classifiers[idx](hidden_cls))
        else:
            out = []
            tokens = self.tokenizer(src, padding=True, truncation=True, max_length=50)
            inputs = torch.LongTensor(tokens['input_ids']).to(device)
            if has_mask == True:
                # attention_mask=(inputs != 0).float() # here `0` is the <pad> token, i guess
                attention_mask = torch.LongTensor(tokens['attention_mask']).to(device)
                pre_train_output = self.pretrain_model(inputs, attention_mask=attention_mask)
            else:
                pre_train_output = self.pretrain_model(inputs)
            # last_hidden = pre_train_output["last_hidden_state"]      # (N,T,E)
            last_hidden = pre_train_output["hidden_states"][self.last_hidden_layer]
            last_hidden_state_cls = last_hidden[:, 0, :].unsqueeze(1)   # (N,1,E)
            last_hidden_state_rest = last_hidden[:, 1:, :] # (N,T-1,E)
            atten_mask_pad = (inputs == 0)[:,1:] #(N,T-1)
            last_attn_out, last_attn_w = self.last_attn(last_hidden_state_cls, last_hidden_state_rest, last_hidden_state_rest,
                                                        key_padding_mask=atten_mask_pad) #(N,1,E), (N,1,T-1)
            last_attn_out = last_attn_out.squeeze(1) #(N,E)
            output = self.final_classifier(last_attn_out)
            out += [output, last_attn_w]
            ## auxiliary_head
            if self.auxiliary_head is not None:
              if "hidden_states" not in pre_train_output:
                raise Exception("Put `pre_train_output=True` in AutoConfig")
              for idx in range(len(self.aux_classifiers)):
                hidden_cls = pre_train_output["hidden_states"][self.auxiliary_head[idx-1]][:, 0, :] # (N,E)
                out.append(self.aux_classifiers[idx](hidden_cls))
        if stage1 == True:
            outlabel = self.stageone(out[0])
            return [outlabel]
        else:
            outlabel = self.stageone(out[0])
            return out, [outlabel]
    
# ## Usage
# tweet = iter(dataloader_train).next()[-1]
# modelA = LastAttnModel(pretrain_model, tokenizer).to(device)
# out, attn_weight = modelA(list(tweet))
# print(out.shape, attn_weight.shape)

In [ ]:
class LanguageModelCNN(nn.Module):
    
    def __init__(self, model_name, num_labels = 3, freeze_pretrained=True):
        super(LanguageModelCNN, self).__init__()
        self.model = AutoModel.from_pretrained(model_name, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, normalization=True)
        
        self.conv = nn.Conv2d(in_channels=13, out_channels=13, kernel_size=(3, 768), padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(624, 3) # before : 442 with max_length 36 # 806 with max_length 64
        self.flat = nn.Flatten()
        self.softmax = nn.LogSoftmax(dim=1)
        if freeze_pretrained:
            print("You are freezing the BERT")
            for name, p in self.model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
#                 if '11' in name:
#                     p.requires_grad = True
#                 if '10' in name:
#                      p.requires_grad = True
#                 if '9' in name:
#                      p.requires_grad = True
        print(f"Total number of params: {number_params(self.model)}")
        print(f"Total number of trainable params: {number_params(self.model, exclude_freeze=True)}")

    def forward(self, src, has_mask=False):
        # print(src)
        slist = []
        for s in src:
            slist.append(" ".join(text_processor.pre_process_doc(s)))
        output = self.tokenizer(slist, padding='max_length', truncation=True, max_length=50)
        output = torch.LongTensor(output['input_ids']).to(device)
        if has_mask == True:
            attention_mask=(output != 0).float() # here `0` is the <pad> token, i guess
            output = self.model(output, attention_mask=attention_mask, output_hidden_states=True)
        else:
            output = self.model(output, output_hidden_states=True)
        
        out = output['hidden_states']
        
        x = torch.transpose(torch.cat(tuple([t.unsqueeze(0) for t in out]), 0), 0, 1)
        # print(x.size())# 32 13 50 768
        x = self.pool(self.dropout(self.relu(self.conv(self.dropout(x)))))
        x = self.dropout(self.flat(self.dropout(x)))
        #print(x.size())
        x = self.fc(x)
        
        return [self.softmax(x)]

In [ ]:
class LanguageModelAllAttn(nn.Module):
    
    def __init__(self, model_name, num_labels = 3, freeze_pretrained=False, last_attn_num_head = 8,):
        super(LanguageModelAllAttn, self).__init__()
        
        self.model = AutoModel.from_pretrained(model_name, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, normalization=True)
        embed_size =self.model.embeddings.word_embeddings.embedding_dim
        # self.conv = nn.Conv2d(in_channels=13, out_channels=13, kernel_size=(3, 768), padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(624, 3) # before : 442 with max_length 36 # 806 with max_length 64
        self.flat = nn.Flatten()
        self.softmax = nn.LogSoftmax(dim=1)
        # 13 transformers for bert 
        self.attn = nn.ModuleList([nn.MultiheadAttention(embed_size, last_attn_num_head, batch_first=True) for i in range(13)])
        self.conv = nn.Conv1d(13, 1, 1, stride=1, padding=0)
        self.linear = nn.Linear(768,3)
        if freeze_pretrained:
            print("You are freezing the BERT")
            for name, p in self.model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
                if '11' in name:
                    p.requires_grad = True
                if '10' in name:
                     p.requires_grad = True
#                 if '9' in name:
#                      p.requires_grad = True
        print(f"Total number of params: {number_params(self.model)}")
        print(f"Total number of trainable params: {number_params(self.model, exclude_freeze=True)}")

    def forward(self, src, has_mask=False):
        # print(src)
        slist = []
        for s in src:
            slist.append(" ".join(text_processor.pre_process_doc(s)))
        output = self.tokenizer(slist, padding='max_length', truncation=True, max_length=50)
        tok = torch.LongTensor(output['input_ids']).to(device)
        
        if has_mask == True:
            attention_mask=(tok != 0).float() # here `0` is the <pad> token, i guess
            output = self.model(tok, attention_mask=attention_mask, output_hidden_states=True)
        else:
            output = self.model(tok, output_hidden_states=True)
        
        out = output['hidden_states']
        
        x = torch.transpose(torch.cat(tuple([t.unsqueeze(0) for t in out]), 0), 0, 1)
        # print(x.size())# 32 13 50 768
        layer_attns = []
        for i in range(x.size(1)):
        
            hidden_state_cls = x[:, i, 0, :].unsqueeze(1)   # (N,i,1,E)
            hidden_state_rest = x[:, i, 1:, :] # (N,i,T-1,E)
            atten_mask_pad = (tok == 0)[:,1:] #(N,T-1)
            last_attn_out, last_attn_w = self.attn[i](hidden_state_cls, hidden_state_rest, hidden_state_rest,
                                                        key_padding_mask=atten_mask_pad) #(N,1,E), (N,1,T-1)
            last_attn_out = last_attn_out.squeeze(1) #(N,E)
            layer_attns.append(last_attn_out)
        layer_outputs = torch.stack(layer_attns)
        layer_outputs = torch.transpose(layer_outputs, 0, 1)
        
        layer_outputs = self.relu(self.conv(layer_outputs).squeeze(1))
        #print(layer_outputs.size())
        layer_outputs = self.linear(layer_outputs)
        #print(layer_outputs.size())
       
        
        
        
        #x = self.pool(self.dropout(self.relu(self.conv(self.dropout(x)))))
        #x = self.dropout(self.flat(self.dropout(x)))
        #print(x.size())
        #x = self.fc(x)
        
        return [self.softmax(layer_outputs)]

In [ ]:
class LanguageModel(nn.Module):
    
    def __init__(self, model_name, num_labels = 3, freeze_pretrained=False):
        super(LanguageModel, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, normalization=True)
        if freeze_pretrained:
            print("You are freezing the BERT")
            for name, p in self.model.named_parameters():
                if 'classifier' not in name:
                    p.requires_grad = False
                if '11' in name:
                     p.requires_grad = True
                if '10' in name:
                     p.requires_grad = True
                if '9' in name:
                     p.requires_grad = True
        print(f"Total number of params: {number_params(self.model)}")
        print(f"Total number of trainable params: {number_params(self.model, exclude_freeze=True)}")

    def forward(self, src, has_mask=False):
        # print(src)
        slist = []
        for s in src:
            slist.append(" ".join(text_processor.pre_process_doc(s)))
        output = self.tokenizer(slist, padding=True, truncation=True, max_length=50)
        output = torch.LongTensor(output['input_ids']).to(device)
        if has_mask == True:
            attention_mask=(output != 0).float() # here `0` is the <pad> token, i guess
            output = self.model(output, attention_mask=attention_mask)
        else:
            output = self.model(output)
        print(output[0].size())
        return output

# Initialization

In [ ]:
from transformers import AutoModel
from transformers import BertModel, BertTokenizer
dataset = Dataset(train_data, model_name)
dataloader_train = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
dataset_valid = Dataset(valid_data, model_name)
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=64, shuffle=False)
dataset_test = Dataset(test_data, model_name)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=64, shuffle=False)
datasetSample = DatasetSample(train_data, model_name, device=device)
dataloadersample = torch.utils.data.DataLoader(datasetSample, batch_size= 1, shuffle=False)

model_name = 'bert-base-uncased'
lossfn = nn.CrossEntropyLoss()
# model = LanguageModel(model_name, freeze_pretrained=False).to(device)
# model = LanguageModelAllAttn(model_name, freeze_pretrained=True,).to(device)
model = LanguageModelCNN(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 30

# Train directly

In [ ]:
def train(dataloader_train, dataloader_valid = None, model = None, 
          optimizer = None, lossfn = None,  epochs = 10, has_mask = True):
    
    trainloss = []
    validloss = []
    trainscore = []
    validscore = []

    for i in range(epochs):
        model.train()
        averageloss = 0
        averagescore = 0
        for datas in tqdm(dataloader_train):
            count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
            optimizer.zero_grad()
            pred = model(list(tweet), has_mask)
            loss = lossfn(pred[0], target.squeeze(1))
            loss.backward()
            optimizer.step()
            f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
            averageloss += loss.item()/len(dataloader_train)
            averagescore += f1score/len(dataloader_train)
        #print(classification_report(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy()))
        trainloss.append(averageloss)
        trainscore.append(averagescore)
        if dataloader_valid is not None:
            model.eval()
            averageloss = 0
            averagescore = 0
            for datas in tqdm(dataloader_valid):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                pred = model(list(tweet), has_mask)
                loss = lossfn(pred[0], target.squeeze(1))
                f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                averageloss += loss.item()/len(dataloader_valid)
                averagescore += f1score/len(dataloader_valid)
            #print(classification_report(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy()))
            validloss.append(averageloss)
            validscore.append(averagescore)
            print(f"epoch: {i}, train loss: {trainloss[-1]}, validation loss: {validloss[-1]}\n train f1score: {trainscore[-1]}, validation f1score: {validscore[-1]}")
        else:
            print(f"epoch: {i}, train loss: {trainloss[-1]}, train f1score: {trainscore[-1]}")

        if (i+1) % 1 == 0:
            torch.save({
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'trainloss': trainloss,
                    'trainscore': trainscore,
                    'validloss': validloss,
                    'validscore': validscore
                    }, './outputmodel_{}'.format(i))
    return trainloss, validloss, trainscore, validscore
            

In [ ]:
# untoggle to train this session
# trainloss, validloss, trainscore, validscore = train(dataloader_train, dataloader_valid, model = model, optimizer = optimizer, 
#       lossfn = lossfn, epochs = epochs, has_mask = True)

In [ ]:

def test(dataloader_valid, model):
    averageloss = 0
    averagescore = 0
    averagePrecision = 0
    averageRecall = 0
    tocsv = []
    with torch.no_grad():
        for datas in tqdm(dataloader_valid):
            count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]

            pred = model(list(tweet), True)
            tocsv.append(pred[0].cpu().numpy()[0])
            f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
            precision=sklearn.metrics.precision_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average='weighted', zero_division=0)
            recall=sklearn.metrics.recall_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(),  average='weighted', zero_division=0)

    #         averageloss += loss.item()/len(dataloader_valid)
            averagescore += f1score/len(dataloader_valid)
            averagePrecision += precision/len(dataloader_valid)
            averageRecall += recall/len(dataloader_valid)

    #     print("averageloss: {}".format(averageloss))
        print("averagescore: {}".format(averagescore))
        print("averagePrecision: {}".format(averagePrecision))
        print("averageRecall: {}".format(averageRecall))

        print("=============")
        return tocsv

# tocsv = test(dataloader_test, model)

In [ ]:
# df = pd.DataFrame(tocsv)
# df.to_csv('./outputmodel_CNN.csv')

# Train Contrastive Learning

In [ ]:
class ContrastiveLoss(nn.Module):

    def __init__(self,):
        super(ContrastiveLoss, self).__init__()
        self.crit =  nn.MSELoss()
        self.crit2 = nn.BCEWithLogitsLoss()
        self.ce =  nn.CrossEntropyLoss()
        self.softmax = nn.Softmax(dim = -1)
    def forward(self, pred, embeddings, target, topk=10, MSELoss = True):
#         print(pred.size()) # 32, 20
#         print(embeddings.size())# 3, 20
#         print(target.size()) # 32
        loss = 0
        simlist = []
        for j in range(embeddings.size(0)):
            # cosineSim = nn.functional.cosine_similarity(pred, embeddings[j].unsqueeze(0), dim=1, eps=1e-08) 
            # size = cosineSim.size()
            simlist.append(pred*embeddings[j].unsqueeze(0))
        cosineSim = torch.stack(simlist) # 3,32, 20
        cosineSim = torch.transpose(cosineSim,0,1) # 32,3,20
        cosineSim = torch.sum(cosineSim, dim= -1)# 32,3
        targetOnehot = nn.functional.one_hot(target, num_classes=3)
        # loss = self.ce(cosineSim, target)
        logits = self.softmax(cosineSim)
        pos = logits*targetOnehot
        pos = torch.sum(pos, dim=-1)
        logitssum = torch.sum(logits, dim=-1)
        pos = -torch.log(pos/logitssum)
#         if topk < pred.size(0):
#             pos = torch.topk(pos, topk)[0]
#             loss = torch.sum(pos)/topk
#         else:
#             loss = torch.sum(pos)/pred.size(0)
        loss = torch.sum(pos)/pred.size(0)
        return loss, logits

In [ ]:
dataset = Dataset(train_data, model_name)
dataloader_train = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
dataset_valid = Dataset(valid_data, model_name)
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
dataset_test = Dataset(test_data, model_name)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)
datasetSample = DatasetSample(train_data, model_name, device=device)
dataloadersample = torch.utils.data.DataLoader(datasetSample, batch_size= 1, shuffle=False)


In [ ]:
from transformers import AutoModel
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer, AutoConfig
# model = LanguageModel(model_name, freeze_pretrained=False).to(device)
# model = LanguageModelAllAttn(model_name, freeze_pretrained=True,).to(device)
# model = LanguageModelCNNForContrastive(model_name).to(device)
lossfn = ContrastiveLoss()
model_name = 'bert-base-uncased'
config = AutoConfig.from_pretrained(
    model_name, 
    output_hidden_states = True,
    output_attention = False,
#     hidden_dropout_prob = 0.2,
) 
pretrain_model = AutoModel.from_pretrained(
    model_name,
    config = config
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LastAttnModelForContrastive(pretrain_model, tokenizer, freeze_pretrained=True,
                       classifier_dropout=0,
                       auxiliary_head=None,
                       last_hidden_layer=-1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 100

# Stage1

In [ ]:
class ContrastiveLoss(nn.Module):

    def __init__(self,):
        super(ContrastiveLoss, self).__init__()
        self.crit =  nn.MSELoss()
        self.crit2 = nn.BCEWithLogitsLoss()
        self.ce =  nn.CrossEntropyLoss()
        self.softmax = nn.Softmax(dim = -1)
    def forward(self, pred, embeddings, target, topk=10, MSELoss = True):
#         print(pred.size()) # 32, 20
#         print(embeddings.size())# 3, 20
#         print(target.size()) # 32
        loss = 0
        simlist = []
        for j in range(embeddings.size(0)):
            # cosineSim = nn.functional.cosine_similarity(pred, embeddings[j].unsqueeze(0), dim=1, eps=1e-08) 
            # size = cosineSim.size()
            simlist.append(pred*embeddings[j].unsqueeze(0))
        cosineSim = torch.stack(simlist) # 3,32, 20
        cosineSim = torch.transpose(cosineSim,0,1) # 32,3,20
        cosineSim = torch.sum(cosineSim, dim= -1)# 32,3
        targetOnehot = nn.functional.one_hot(target, num_classes=3)
        # loss = self.ce(cosineSim, target)
        logits = self.softmax(cosineSim)
        pos = logits*targetOnehot
        pos = torch.sum(pos, dim=-1)
        logitssum = torch.sum(logits, dim=-1)
        pos = -torch.log(pos/logitssum)
#         if topk < pred.size(0):
#             pos = torch.topk(pos, topk)[0]
#             loss = torch.sum(pos)/topk
#         else:
#             loss = torch.sum(pos)/pred.size(0)
        loss = torch.sum(pos)/pred.size(0)
        return loss, logits

In [ ]:
lossfn = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 30

In [ ]:
# run this code to train stage one.
count = 0
compareIter = 10
trainloss = []
validloss = []
trainscore = []
validscore = []
trainscoreCE = []
validscoreCE = []
for i in range(epochs):
    averagescore = 0
    averagescoreCE = 0
    averageloss = 0
    for datas in tqdm(dataloader_train):
        model.train()
        count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
        optimizer.zero_grad()
#         with torch.no_grad():
        pred, spec = model(list(tweet), True)
        ce =  nn.CrossEntropyLoss()
        # loss1 = ce(spec[0],  target.squeeze(1))
        spec = torch.argmax(spec[0], dim=1)
        spec = spec.cpu().detach().numpy()
        pred = pred[0]
            # print(pred.size())
        argmaxs = []
        loss = 0
        logitslist = []
        for k in range(compareIter):
            sampledatas = next(iter(dataloadersample))
            inputsample = []
            for j in range(len(sampledatas)): #3
                inputsample.append(sampledatas[j][0])
            #with torch.no_grad():
            pred1, spec1 = model(list(inputsample), True)
            pred1 = pred1[0]
            loss, logits = lossfn(pred, pred1, target.squeeze(1))
            logitslist.append(logits)
            loss += loss/compareIter
        loss = loss
        logitslist = torch.stack(logitslist)
        logitslist = torch.sum(logitslist, dim=0) # 32,3
        logitslist = torch.argmax(logitslist, dim=1)
        argmaxs = logitslist.cpu().detach().numpy()
        loss.backward()
        optimizer.step()
        
        f1score = sklearn.metrics.f1_score( target.squeeze(1).cpu().numpy(), argmaxs, average = 'weighted')
        f1score_ce = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), spec, average = 'weighted')
#         print(f1score)
        averageloss += loss.item()/len(dataloader_train)
        averagescore += f1score/len(dataloader_train)
        averagescoreCE += f1score_ce/len(dataloader_train)
#     print(averagescore)
#     print(averageloss)
    trainloss.append(averageloss)
    trainscore.append(averagescore)
    trainscoreCE.append(averagescoreCE)
    averagescore = 0
    averageloss = 0
    averagescoreCE = 0
    for datas in tqdm(dataloader_valid):
        
        model.eval()
        count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
        with torch.no_grad():
            pred, spec = model(list(tweet), True)
            ce =  nn.CrossEntropyLoss()
            loss1 = ce(spec[0],  target.squeeze(1))
            spec = torch.argmax(spec[0], dim=1)
            spec = spec.cpu().detach().numpy()
            pred = pred[0]
            argmaxs = []
            loss = 0
            logitslist = []
            for k in range(compareIter):
                sampledatas = next(iter(dataloadersample))
                inputsample = []
                for j in range(len(sampledatas)): #3
                    inputsample.append(sampledatas[j][0])

                loss, logits = lossfn(pred, model(list(inputsample), True)[0][0], target.squeeze(1))
                logitslist.append(logits)
                loss += loss/compareIter
            loss = loss
            logitslist = torch.stack(logitslist)
            logitslist = torch.sum(logitslist, dim=0) # 32,3
            logitslist = torch.argmax(logitslist, dim=1)
            argmaxs = logitslist.cpu().detach().numpy()
            
            f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), argmaxs, average = 'weighted')
            f1score_ce = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), spec, average = 'weighted')
    #         print(f1score)
            averageloss += loss.item()/len(dataloader_valid)
            averagescore += f1score/len(dataloader_valid)
            averagescoreCE += f1score_ce/len(dataloader_valid)
    validloss.append(averageloss)
    validscore.append(averagescore)
    validscoreCE.append(averagescoreCE)
    if (i+1) % 1 == 0:
        torch.save({
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'trainloss': trainloss,
                'trainscore': trainscore,
                'validloss': validloss,
                'validscore': validscore
                }, './outputmodel_{}'.format(i))
    print(f"epoch: {i}, train loss: {trainloss[-1]}, validation loss: {validloss[-1]}\n train f1score: {trainscore[-1]}, validation f1score: {validscore[-1]}")




In [ ]:
# checkpoint = torch.load('../input/outputmodel10k/outputmodel_0')
# model.load_state_dict(checkpoint['model'], strict=False)

In [ ]:
# for k=10, outputmodel10 is the best
compareIter = 10
def test_stage1(): 
    averagescore = 0
    for datas in tqdm(dataloader_valid):
        model.eval()
        count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
        with torch.no_grad():
            pred, spec = model(list(tweet), True)
            ce =  nn.CrossEntropyLoss()
            loss1 = ce(spec[0],  target.squeeze(1))
            spec = torch.argmax(spec[0], dim=1)
            spec = spec.cpu().detach().numpy()
            pred = pred[0]
            argmaxs = []
            loss = 0
            logitslist = []
            for k in range(compareIter):
                sampledatas = next(iter(dataloadersample))
                inputsample = []
                for j in range(len(sampledatas)): #3
                    inputsample.append(sampledatas[j][0])

                loss, logits = lossfn(pred, model(list(inputsample), True)[0][0], target.squeeze(1))
                logitslist.append(logits)
                loss += loss/compareIter
            loss = loss
            logitslist = torch.stack(logitslist)
            logitslist = torch.sum(logitslist, dim=0) # 32,3
            logitslist = torch.argmax(logitslist, dim=1)
            argmaxs = logitslist.cpu().detach().numpy()
            
            f1score = sklearn.metrics.f1_score( target.squeeze(1).cpu().numpy(), argmaxs, average = 'weighted')
            averagescore += f1score/len(dataloader_valid)
    print(averagescore)
    
# for i in range(20):
#     checkpoint = torch.load('../input/outputmodel10k/outputmodel_{}'.format(i))
#     model.load_state_dict(checkpoint['model'], strict=False)
#     test_stage1()

# Stage2

In [ ]:
class SoftCrossEntropyLoss(nn.Module):
    def __init__(self, weights):
        super().__init__()
        self.weights = weights

    def forward(self, y_hat, y):
        p = F.log_softmax(y_hat, 1)
        w_labels = self.weights*y
        loss = -(w_labels*p).sum() / (w_labels).sum()
        return loss


In [ ]:

lossfn = SoftCrossEntropyLoss(1)
lossfn = nn.CrossEntropyLoss()
epochs = 40
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# can load pretrain model from stage1
# checkpoint = torch.load('./outputmodel_11')
# model.load_state_dict(checkpoint['model'], strict=False)

In [ ]:
def train(dataloader_train, dataloader_valid = None, model = None, 
          optimizer = None, lossfn = None,  epochs = 10, has_mask = True):
    
    trainloss = []
    validloss = []
    trainscore = []
    validscore = []

    for i in range(epochs):
        model.train()
        averageloss = 0
        averagescore = 0
        for datas in tqdm(dataloader_train):
            count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
            smoothTarget = torch.stack([hate_speech.squeeze(-1), offensive_language.squeeze(-1), neither.squeeze(-1)])
            smoothTarget = torch.transpose(smoothTarget,  0, 1)
            sumSmooth = torch.sum(smoothTarget,dim=-1).unsqueeze(1)
            smoothTarget = smoothTarget/sumSmooth
            
            optimizer.zero_grad()
            pred = model(list(tweet), has_mask, stage1 = True, nograd = True)
            loss = lossfn(pred[0], target.squeeze(1))
            loss.backward()
            optimizer.step()
            f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
            averageloss += loss.item()/len(dataloader_train)
            averagescore += f1score/len(dataloader_train)
        #print(classification_report(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy()))
        trainloss.append(averageloss)
        trainscore.append(averagescore)
        if dataloader_valid is not None:
            model.eval()
            averageloss = 0
            averagescore = 0
            for datas in tqdm(dataloader_valid):
                count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
                smoothTarget = torch.stack([hate_speech.squeeze(-1), offensive_language.squeeze(-1), neither.squeeze(-1)])
                smoothTarget = torch.transpose(smoothTarget,  0, 1)
                sumSmooth = torch.sum(smoothTarget,dim=-1).unsqueeze(1)
                smoothTarget = smoothTarget/sumSmooth

                pred = model(list(tweet), has_mask, stage1 = True)
                loss = lossfn(pred[0], target.squeeze(1))
                f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
                averageloss += loss.item()/len(dataloader_valid)
                averagescore += f1score/len(dataloader_valid)
            #print(classification_report(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy()))
            validloss.append(averageloss)
            validscore.append(averagescore)
            print(f"epoch: {i}, train loss: {trainloss[-1]}, validation loss: {validloss[-1]}\n train f1score: {trainscore[-1]}, validation f1score: {validscore[-1]}")
        else:
            print(f"epoch: {i}, train loss: {trainloss[-1]}, train f1score: {trainscore[-1]}")
        if (i+1) % 1 == 0 and i >5:
            torch.save({
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'trainloss': trainloss,
                    'trainscore': trainscore,
                    'validloss': validloss,
                    'validscore': validscore
                    }, './outputmodel_{}'.format(i))

    return trainloss, validloss, trainscore, validscore
            

In [ ]:
trainloss1, validloss1, trainscore1, validscore1 = train(dataloader_train, dataloader_valid, model = model, optimizer = optimizer, 
      lossfn = lossfn, epochs = epochs, has_mask = True)

# Test contrastive

In [ ]:
!ls ../input/nlpmodel/
dataset = Dataset(train_data, model_name)
dataloader_train = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
dataset_valid = Dataset(valid_data, model_name)
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=32, shuffle=False)
dataset_test = Dataset(test_data, model_name)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False)
datasetSample = DatasetSample(train_data, model_name, device=device)
dataloadersample = torch.utils.data.DataLoader(datasetSample, batch_size= 1, shuffle=False)

# can load model after training
# checkpoint = torch.load('../input/bugfixnlpmodel/outputmodel_3610knewtune')
# checkpoint = torch.load('./outputmodel_19')
# model.load_state_dict(checkpoint['model'])

In [ ]:
def test(dataloader_valid, model):
    averageloss = 0
    averagescore = 0
    averagePrecision = 0
    averageRecall = 0
    tocsv = []
    targettocsv = []
    with torch.no_grad():
        for datas in tqdm(dataloader_valid):
            count, hate_speech, offensive_language, neither, target, tweet = datas[0], datas[1], datas[2], datas[3], datas[4], datas[5]
            smoothTarget = torch.stack([hate_speech.squeeze(-1), offensive_language.squeeze(-1), neither.squeeze(-1)])
            smoothTarget = torch.transpose(smoothTarget,  0, 1)
            sumSmooth = torch.sum(smoothTarget,dim=-1).unsqueeze(1)
            smoothTarget = smoothTarget/sumSmooth

            pred = model(list(tweet), True, stage1=True)
#             print(pred[0])
            tocsv.append(pred[0].cpu().numpy()[0])
            targettocsv.append(target.squeeze(1).cpu().numpy())
            f1score = sklearn.metrics.f1_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average = 'weighted')
            precision=sklearn.metrics.precision_score( target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average='weighted', zero_division=0)
            recall=sklearn.metrics.recall_score(target.squeeze(1).cpu().numpy(), pred[0].argmax(-1).cpu().numpy(), average='weighted', zero_division=0)

    #         averageloss += loss.item()/len(dataloader_valid)
            averagescore += f1score/len(dataloader_valid)
            averagePrecision += precision/len(dataloader_valid)
            averageRecall += recall/len(dataloader_valid)

    #     print("averageloss: {}".format(averageloss))
        print("averagescore: {}".format(averagescore))
        print("averagePrecision: {}".format(averagePrecision))
        print("averageRecall: {}".format(averageRecall))

        print("=============")
    return tocsv, targettocsv
# for i in range(33):
#     print(i+6)
#     checkpoint = torch.load('./outputmodel_{}'.format(i+6))
#     model.load_state_dict(checkpoint['model'], strict=True)
tocsv, targettocsv = test(dataloader_test, model)

In [ ]:
# df = pd.DataFrame(tocsv)
# df.to_csv('./outputmodel_CLnew.csv')
# df = pd.DataFrame(targettocsv)
# df.to_csv('./target.csv')

# Ensemble

In [ ]:
files = ['../input/nlpcsvs/bi_lstm.csv',
          '../input/nlpcsvs/lastATNN.csv','../input/nlpcsvs/lastATNN_allCLS.csv', './outputmodel_CL.csv']
# 0.900, 0.893, 0.901, 0.900
files = ['./outputmodel_CLnew.csv'] 
def combine(files = []):
    softmax = nn.Softmax(dim=-1)
    for i in range(len(files)):
        df = pd.read_csv(files[i] ,index_col = 0)
        df = torch.tensor(df.values)
        df = softmax(df)
        if i > 0:
            df_combine += df
        if i == 0:
            df_combine = df
    df_combine = df_combine.argmax(-1).numpy()
    return df_combine
df_combine = combine(files)
target = pd.read_csv('./target.csv', index_col = 0)

In [ ]:
f1score = sklearn.metrics.f1_score(target, df_combine, average = 'weighted')
precision = sklearn.metrics.precision_score(target, df_combine, average='weighted', zero_division=0)
recall = sklearn.metrics.recall_score(target, df_combine, average='weighted', zero_division=0)
print(f1score)
print(precision)
print(recall)
